In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import glob
import seaborn as sns
import numpy as np
import os
import sys
import torch
from PIL import Image
from torchvision import transforms
from torch.multiprocessing import Pool, Process, set_start_method
import csv
try:
    set_start_method('spawn')
except RuntimeError:
    pass
sys.path.append('/home/lincolnzjx/Desktop/Interpretation/src/utils/')
sys.path.append('/home/lincolnzjx/Desktop/Interpretation/src/')
sys.path.append('/home/lincolnzjx/Desktop/Interpretation/jupyter/')
from function import gcollect
from datasets import imagenet
from cal_metric import main, writer
import model

## 伪代码

1. 遍历文件夹，发现所有的类别
2. 对于每一个类别的每一张opt中的图片计算对应的数字
3. 保存到每一个文件夹下的csv文件夹下面

In [5]:
dataset_path = "/media/lincolnzjx/HardDisk/Datasets/ilsvrc2012/train/"
dir_class_path = "/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/GBP/*"
device = torch.device('cpu')
num_class = 30
con_run_process = 10

In [6]:
image2index = {}
for real_path in glob.glob(dir_class_path):
    class_index = real_path.split('/')[-1]
    class_index = int(class_index)
    train_transform = transforms.Compose([
                transforms.Resize((224, 224), interpolation=Image.BILINEAR),
                transforms.ToTensor()
            ])
    trainset = imagenet.ImageNet(root="/media/lincolnzjx/HardDisk/" +
                                 "Datasets/",
                                 is_train=True,
                                 transform=train_transform)
    trainset.set_data([class_index], num_class)
    
    for index, (_, _, img_path) in enumerate(trainset):
        img_name = img_path.split('/')[-1].split('.')[0]
        image2index[img_name] = index

Len of new dataset is :30
Len of new dataset is :30
Len of new dataset is :30
Len of new dataset is :30
Len of new dataset is :30
Len of new dataset is :30


## GBP

In [41]:
## GBP
dir_class_path = "/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/GBP/*"
dir_path = "/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/GBP/*/opt/*"


processes = Pool(processes=con_run_process)
for real_path in glob.glob(dir_path):
    img_name = real_path.split('/')[-1]
    img_prefix_path = real_path.rsplit('/', maxsplit=2)[0]
    class_index = img_prefix_path.split('/')[-1]
    # if int(class_index) != 522:
    #     continue
    img_prefix_path = os.path.join(img_prefix_path, 'csv')
    os.makedirs(img_prefix_path, exist_ok=True)
    _, selected_layer, _, selected_filter, class_name, class_img_index = img_name.split('_')
    
    ori_path = os.path.join(dataset_path, class_name, 
                            class_name+'_'+class_img_index)
    ori_path = ori_path.replace('png', 'JPEG')
    opt_path = real_path
    
    real_img_name = (class_name+'_'+class_img_index).replace('.png', '')
    if real_img_name not in image2index:
        continue
    writer_path = os.path.join(img_prefix_path, '.'.join((str(class_index)+ 
                                                          str(image2index[real_img_name]), 
                                                          selected_layer, 
                                                          selected_filter, 
                                                          'gbp'+'.csv')))
    selected_layer = int(selected_layer)
    selected_filter = int(selected_filter)
    print(ori_path)
    print(opt_path)
    print(writer_path)
        
    backbone = 'vgg16'
    net = model.Network(backbone=backbone, num_classes=1000,
                        selected_layer=selected_layer)
    # resume from model
    resume = "037-0"
    model_dir = "/home/lincolnzjx/Desktop/Interpretation/saved/models"
    resume_exp = resume.split("-")[0]
    resume_epoch = resume.split("-")[1]
    resume_path = os.path.join(model_dir, str(resume_exp),
                               str(resume_epoch))
    ckpt = torch.load(resume_path, map_location=device)
    net.load_state_dict(ckpt, strict=False)
    net.to(device)
    net.eval()
    # sel_error, res_error = main(selected_layer, selected_filter, net, 
    #                             ori_path, opt_path)
    processes.apply_async(main, args=(selected_layer,
                                      selected_filter,
                                      net, 
                                      ori_path, opt_path,
                                      writer, writer_path))
processes.close()
processes.join()

/media/lincolnzjx/HardDisk/Datasets/ilsvrc2012/train/n03134739/n03134739_1008.JPEG
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/GBP/522/opt/layer_25_filter_391_n03134739_1008.png
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/GBP/522/csv/5228.25.391.gbp.csv
Only Keep 25th layers before.
/media/lincolnzjx/HardDisk/Datasets/ilsvrc2012/train/n03134739/n03134739_10205.JPEG
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/GBP/522/opt/layer_1_filter_57_n03134739_10205.png
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/GBP/522/csv/52219.1.57.gbp.csv
Only Keep 1th layers before.
/media/lincolnzjx/HardDisk/Datasets/ilsvrc2012/train/n03134739/n03134739_10266.JPEG
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/GBP/522/opt/layer_11_filter_211_n03134739_10266.png
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/GBP/522/csv/52227.11.211.gbp.csv
Only Keep 11th layers before.
/media/lincolnzjx/HardDisk/Datasets/ilsvrc201

KeyboardInterrupt: 

In [31]:
pwd

'/home/lincolnzjx/Desktop/Interpretation/jupyter'

## Ours

In [15]:
excel_path = "./twy_exps.xlsx"
sheet = "twy"
excepts = []

In [24]:
xlsx = pd.ExcelFile(excel_path)
excel = pd.read_excel(xlsx, sheet, dtype={"exp": str, 
                                          "Done": str,
                                          "selected_layer": int,
                                          "selected_filter": int, 
                                          "Alpha": int,
                                          "Beta": int,
                                          "Gamma": int,
                                          "cuda": int,
                                          "class_index": int,
                                          "num_class": int,
                                          "re_size": int,
                                          "dataset": str,
                                          "backbone": str, 
                                          "mode":str})
excel = excel.set_index('exp')

In [42]:
## twy, ours
# dir_class_path = "/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/twy/*/_best.png"
do_list = {
#     522: ['29.292'], 
         84: ['27.328']}
#          950: ['25.1', '22.164', '6.76']}
dir_path = "/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/twy/*/*_best.png"
processes = Pool(processes=con_run_process)
for real_path in glob.glob(dir_path):
    img_name = real_path.split('/')[-1]
    exp = 'x' + real_path.split('/')[-2]
    if exp not in excel.index.to_list():
        continue
    img_prefix_path = real_path.rsplit('/', maxsplit=2)[0] + '/csv/'
    
    selected_layer = excel.loc[exp, 'selected_layer']
    selected_filter = excel.loc[exp, 'selected_filter']
    class_index = excel.loc[exp, 'class_index']
    
    if not (int(class_index) in do_list and '.'.join([str(selected_layer), str(selected_filter)]) in do_list[class_index]):
        continue
    print(real_path)
    print(img_name)
    print(img_prefix_path)
     
    # img_prefix_path = os.path.join(img_prefix_path, 'csv')
    os.makedirs(img_prefix_path, exist_ok=True)
    
    class_name, class_img_index, *_ = img_name.rsplit('_')
    ori_path = os.path.join(dataset_path, class_name, 
                            class_name + '_' + class_img_index + '.JPEG')
    # ori_path = ori_path.replace('png', 'JPEG')
    opt_path = real_path
    
    real_img_name = (class_name+'_'+class_img_index)
    if real_img_name not in image2index:
        print('real', real_img_name)
        continue
    writer_path = os.path.join(img_prefix_path, '.'.join((str(class_index)+ 
                                                          str(image2index[real_img_name]), 
                                                          str(selected_layer), 
                                                          str(selected_filter), 
                                                          'ours'+'.csv')))
    selected_layer = int(selected_layer)
    selected_filter = int(selected_filter)
    print(ori_path)
    print(opt_path)
    print(writer_path)
        
    backbone = 'vgg16'
    net = model.Network(backbone=backbone, num_classes=1000,
                        selected_layer=selected_layer)
    # resume from model
    resume = "037-0"
    model_dir = "/home/lincolnzjx/Desktop/Interpretation/saved/models"
    resume_exp = resume.split("-")[0]
    resume_epoch = resume.split("-")[1]
    resume_path = os.path.join(model_dir, str(resume_exp),
                               str(resume_epoch))
    ckpt = torch.load(resume_path, map_location=device)
    net.load_state_dict(ckpt, strict=False)
    net.to(device)
    net.eval()
    # sel_error, res_error = main(selected_layer, selected_filter, net, 
    #                             ori_path, opt_path)
    processes.apply_async(main, args=(selected_layer,
                                      selected_filter,
                                      net, 
                                      ori_path, opt_path,
                                      writer, writer_path))
processes.close()
processes.join()

/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/twy/101939/n01806143_10265_9822_best.png
n01806143_10265_9822_best.png
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/twy/csv/
/media/lincolnzjx/HardDisk/Datasets/ilsvrc2012/train/n01806143/n01806143_10265.JPEG
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/twy/101939/n01806143_10265_9822_best.png
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/twy/csv/8420.27.328.ours.csv
Only Keep 27th layers before.
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/twy/101939/n01806143_10215_9953_best.png
n01806143_10215_9953_best.png
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/twy/csv/
/media/lincolnzjx/HardDisk/Datasets/ilsvrc2012/train/n01806143/n01806143_10215.JPEG
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/twy/101939/n01806143_10215_9953_best.png
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/twy/csv/8416.27.328.ours.csv
Only Keep 27th

Only Keep 27th layers before.
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/twy/101939/n01806143_10028_9992_best.png
n01806143_10028_9992_best.png
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/twy/csv/
/media/lincolnzjx/HardDisk/Datasets/ilsvrc2012/train/n01806143/n01806143_10028.JPEG
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/twy/101939/n01806143_10028_9992_best.png
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/twy/csv/846.27.328.ours.csv
Only Keep 27th layers before.
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/twy/101939/n01806143_10185_9894_best.png
n01806143_10185_9894_best.png
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/twy/csv/
/media/lincolnzjx/HardDisk/Datasets/ilsvrc2012/train/n01806143/n01806143_10185.JPEG
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/twy/101939/n01806143_10185_9894_best.png
/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/twy/csv/8413.2

In [23]:
excel.index.to_list()

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155]

In [67]:
excel

,Done,selected_layer,selected_filter,Alpha,Beta,Gamma,cuda,class_index,num_class,re_size,dataset,backbone,mode,inter,defensed
exp,,,,,,,,,,,,,,,
x102117,N,1,16,1,1,1,0,950,30,224,ImageNet,vgg16,single,False,False
x102118,N,3,20,1,1,1,0,950,30,224,ImageNet,vgg16,single,False,False
x102120,N,8,99,1,1,1,5,950,30,224,ImageNet,vgg16,single,False,False
x102121,N,11,75,1,1,1,5,950,30,224,ImageNet,vgg16,single,False,False
x102122,N,13,98,1,1,1,2,950,30,224,ImageNet,vgg16,single,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
x101976,N,20,353,1,1,1,4,14,30,224,ImageNet,vgg16,single,False,False
x101977,N,22,116,1,1,1,9,14,30,224,ImageNet,vgg16,single,False,False
x101978,N,25,407,1,1,1,9,14,30,224,ImageNet,vgg16,single,False,False


In [43]:
sum_num = 0
counter = []

for class_index in [444,14, 522, 84, 948, 950]:
    sel_lay_fil = excel[excel['class_index' ] == class_index][['selected_layer', 'selected_filter']].values
    sel_lay_fil_to_str = ['.'.join(list(map(str, x))) for x in sel_lay_fil]
    sel_lay_fil_to_str = set(sel_lay_fil_to_str)
    # print(set(sel_lay_fil_to_str))
    
    ress = []
    for sel_lay_fil  in sel_lay_fil_to_str:
        sel_lay, sel_fil = sel_lay_fil.split('.')
        
#         print(sel_lay, sel_fil)
        search_path = '/media/lincolnzjx/HardDisk/myGithub/Interpretation/generated/twy/csv/' + str(class_index) + '*.' + sel_lay_fil + '.ours.csv' 
        result = list(glob.glob(search_path))
#         print(result)
#         break
        if len(result) >= 30:
            # print(class_index, img_index, len(result))
            # sum_num = sum_num + 30 - len(result)
            # for res in result:
            #    sel_lay, sel_fil = res.split('/')[-1].split('.')[-4:-2]
            ress.append(sel_lay_fil)
    # print(ress)
    # print(sel_lay_fil_to_str)
    # break
    ress = set(ress)
    ress = sel_lay_fil_to_str - ress
    print(class_index, ress, len(ress))

444 set() 0
14 set() 0
522 set() 0
84 set() 0
948 set() 0
950 set() 0


In [ ]:
# 101939

In [107]:
sel_lay_fil_to_str

{'1.16',
 '1.47',
 '11.174',
 '11.75',
 '13.21',
 '13.98',
 '15.102',
 '15.173',
 '18.353',
 '18.458',
 '20.103',
 '20.46',
 '22.164',
 '22.214',
 '25.1',
 '25.19',
 '27.161',
 '27.476',
 '29.101',
 '29.64',
 '3.20',
 '3.41',
 '6.114',
 '6.76',
 '8.17',
 '8.99'}